In [1]:
def try_download(url):
    response = requests.get(url)
    if response.status_code != 200:
        raise Exception(r"Didn't work, response.status_code = " + str(response.status_code) + ", url = " + url)

    return response

In [2]:
def get_formatted_velocities(config, river_point_config, date_list, z_grid, idx_max_z_river):
    url_discharge = (f"https://alplakes-internal-api.eawag.ch/bafu/hydrodata/measured/{river_point_config.bafu_id}/"
           f"{river_point_config.discharge_variable_name}/"
           f"{config.start_date}/"
           f"{config.end_date}/"
           "?resample=hourly"
           )
    response = try_download(url_discharge)
    json_discharge = response.json()
    discharge_data = parse_river_json(json_discharge)

    formatted_discharge = discharge_data.interp(time=date_list, method='linear')

    river_angle = river_point_config.angle_from_north_direction[0] + config.rotation
    u_flow = formatted_discharge.data.values * np.sin(np.deg2rad(river_angle))
    v_flow = formatted_discharge.data.values * np.cos(np.deg2rad(river_angle))


    depth_profile = np.insert(z_grid[0:(idx_max_z_river+1)], 0, 0)

    uvel_profile = []
    vvel_profile = []
    for i_time, date in enumerate(date_list):
        uvel_profile.append(vectorFlow(u_flow[i_time], depth_profile) / config.grid_resolution)
        vvel_profile.append(vectorFlow(v_flow[i_time], depth_profile) / config.grid_resolution)

    return uvel_profile, vvel_profile

In [3]:
def get_formatted_temperature(config, river_point_config, date_list):
    url_temperature = (f"https://alplakes-internal-api.eawag.ch/bafu/hydrodata/measured/{river_point_config.bafu_id}/"
       f"{river_point_config.temperature_variable_name}/"
       f"{config.start_date}/"
       f"{config.end_date}/"
       "?resample=hourly"
       )
    response = try_download(url_temperature)
    json_temperature = response.json()
    temperature_data = parse_river_json(json_temperature)

    formatted_temperature = temperature_data.interp(time=date_list, method='linear')

    return formatted_temperature.data.values

In [4]:
def get_boundaries(river_point_config, u_mean, v_mean):
    boundaries = []
    if river_point_config.in_or_out == 'in':
        if u_mean < 0:
            boundaries.append('east')
        elif u_mean > 0:
            boundaries.append('west')
        if v_mean < 0:
            boundaries.append('north')
        elif v_mean > 0:
            boundaries.append('south')
    elif river_point_config.in_or_out == 'out':
        if u_mean < 0:
            boundaries.append('west')
        elif u_mean > 0:
            boundaries.append('east')
        if v_mean < 0:
            boundaries.append('south')
        elif v_mean > 0:
            boundaries.append('north')
    else:
        raise ValueError('river_point_config.in_or_out must be "in" or "out"')

    return boundaries

In [5]:
# https://github.com/leroquan/mitgcm_toolbox/blob/master/PythonScripts/generate_river_data.py

from grid_and_bathy import convert_point_coord_to_mitgcm_coord, get_dz_grid
from configs.config_object import ConfigObject
import numpy as np
import os
import pandas as pd
import requests
from generate_river_data import vectorFlow, parse_river_json
from utils import modify_arguments

In [6]:
def build_river_dict(river_dicts, config, river_point_config, z_grid, date_list):
    coord_point = river_point_config.coordinates_lake_point[0]
    x_coord, y_coord = convert_point_coord_to_mitgcm_coord(coord_point[0], coord_point[1], "2056", config)
    x_idx = int(x_coord / config.grid_resolution)+1
    y_idx = int(y_coord / config.grid_resolution)+1

    idx_max_z_river = np.abs(z_grid - river_point_config.river_depth).argmin()

    uvel, vvel = get_formatted_velocities(config, river_point_config, date_list, z_grid, idx_max_z_river)

    river_dicts.append({
        "name": river_point_config.name,
        "x_idx": x_idx,
        "y_idx": y_idx,
        "max_z_idx": idx_max_z_river,
        "boundaries": get_boundaries(river_point_config, np.mean(uvel), np.mean(vvel)),
        "u_velocity": uvel,
        "v_velocity": vvel,
        "temperature": get_formatted_temperature(config, river_point_config, date_list)
    })

    return river_dicts

In [7]:
def define_direction_specific_variables(boundary_direction, config):
    boundary_length = 0
    vel_var = ''
    idx_var = ''
    prefix = ''
    idx_ortho=''
    if boundary_direction == 'north' or boundary_direction == 'south':
        boundary_length = config.Nx
        vel_var = 'v'
        idx_var = 'x_idx'
        prefix = 'J'
        idx_ortho = 'y_idx'
    elif boundary_direction == 'east' or boundary_direction == 'west':
        boundary_length = config.Ny
        vel_var = 'u'
        idx_var = 'y_idx'
        prefix = 'I'
        idx_ortho = 'x_idx'

    return boundary_length, vel_var, idx_var, prefix, idx_ortho

In [8]:
def write_river_binaries(binary_data_folder, boundary_direction, vel_var,
                         date_list, dz_grid, boundary_length, river_dicts, idx_var):
    vel_file = open(f'{binary_data_folder}/bc_{boundary_direction}_{vel_var}.bin', 'ab')
    temperature_file = open(f'{binary_data_folder}/bc_{boundary_direction}_temp.bin', 'ab')

    for i_time, date in enumerate(date_list):
        velocities_bc = np.zeros((len(dz_grid), boundary_length))
        temperatures_bc = np.zeros((len(dz_grid), boundary_length))
        for dict in river_dicts:
            if boundary_direction in dict["boundaries"]:
                vel = dict[f"{vel_var}_velocity"][i_time]
                temp = dict['temperature'][i_time]

                velocities_bc[0:(dict["max_z_idx"]+1), dict[idx_var]] = vel
                temperatures_bc[0:(dict["max_z_idx"]+1), dict[idx_var]] = temp

        velocities_bc.tofile(vel_file)
        temperatures_bc.tofile(temperature_file)

    vel_file.close()
    temperature_file.close()

In [9]:
def build_river_files(config: ConfigObject, output_folder: str, save_files=True):
    binary_data_folder = os.path.join(output_folder, 'binary_data')
    obcs_file_path = os.path.join(output_folder, 'run_config', 'data.obcs')
    date_list = pd.date_range(config.start_date, config.end_date, freq="1h")

    dz_grid = get_dz_grid(os.path.join(config.paths.grid_folder_path, 'dz.csv')).flatten()
    z_grid = np.cumsum(dz_grid)

    river_dicts = []
    for river_point_config in config.rivers:
        river_dicts = build_river_dict(river_dicts, config, river_point_config, z_grid, date_list)

    river_string = ''
    for boundary_direction in ['north', 'south', 'east', 'west']:
        (boundary_length, vel_var,
         idx_var, prefix, idx_ortho) = define_direction_specific_variables(boundary_direction, config)

        discharge_exist=False
        for dict in river_dicts:
            if boundary_direction in dict["boundaries"]:
                discharge_exist=True
                river_string += f' OB_{prefix}{boundary_direction}({dict[idx_var]}) = {dict[idx_ortho]}\n'

        if discharge_exist and save_files:
            write_river_binaries(binary_data_folder, boundary_direction, vel_var,
                         date_list, dz_grid, boundary_length, river_dicts, idx_var)

    if save_files:
        modify_arguments('!set_obs!', river_string[:-1], obcs_file_path)

    return river_dicts, river_string

In [10]:
config = ConfigObject('..//config.json')
output_folder = f'./99-output/geneva_200m_2025/binary_data'
river_dicts, river_string = build_river_files(config, output_folder, False)

In [11]:
river_dicts[0]

{'name': ('south-west-corner',),
 'x_idx': 2,
 'y_idx': 1,
 'max_z_idx': np.int64(5),
 'boundaries': ['west', 'south'],
 'u_velocity': [array([0.1654956 , 0.1639687 , 0.16208082, 0.15958004, 0.15579277,
         0.14596376]),
  array([0.14649029, 0.14496339, 0.1430755 , 0.14057473, 0.13678745,
         0.12695845]),
  array([0.14349854, 0.14197164, 0.14008376, 0.13758298, 0.13379571,
         0.1239667 ]),
  array([0.1416071 , 0.1400802 , 0.13819232, 0.13569154, 0.13190427,
         0.12207526]),
  array([0.13716041, 0.13563351, 0.13374562, 0.13124485, 0.12745757,
         0.11762857]),
  array([0.13169524, 0.13016834, 0.12828046, 0.12577968, 0.12199241,
         0.1121634 ]),
  array([0.12638467, 0.12485777, 0.12296988, 0.12046911, 0.11668183,
         0.10685283]),
  array([0.13099505, 0.12946815, 0.12758026, 0.12507949, 0.12129221,
         0.11146321]),
  array([0.13599644, 0.13446955, 0.13258166, 0.13008088, 0.12629361,
         0.1164646 ]),
  array([0.13128604, 0.12975914, 0.127